# 📊 Datathon FIAP - Passos Mágicos
## Análise Exploratória de Dados (EDA)

**Objetivo:** Realizar análise exploratória completa dos dados do PEDE (Pesquisa Extensiva do Desenvolvimento Educacional) dos anos 2022, 2023 e 2024, respondendo às 11 perguntas de negócio propostas.

**Autores:** Grupo Datathon FIAP

---

## 1. Configuração do Ambiente

In [ ]:
# Montar Google Drive (executar apenas no Google Colab)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalar bibliotecas necessárias
!pip install openpyxl plotly kaleido -q

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Cores personalizadas para as Pedras
CORES_PEDRAS = {
    'Quartzo': '#E8E8E8',
    'Ágata': '#9370DB',
    'Agata': '#9370DB',
    'Ametista': '#8B008B',
    'Topázio': '#FFD700'
}

print('✅ Bibliotecas importadas com sucesso!')

## 2. Carregamento dos Dados

In [ ]:
# Definir caminho do arquivo
# IMPORTANTE: Ajuste o caminho conforme a localização do arquivo no seu Google Drive
CAMINHO_ARQUIVO = '/content/drive/MyDrive/Datathon/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'

# Alternativa: Se o arquivo estiver em outro local, descomente e ajuste:
# CAMINHO_ARQUIVO = '/content/BASE_DE_DADOS_PEDE_2024_DATATHON.xlsx'

# Carregar todas as abas do Excel
xlsx = pd.ExcelFile(CAMINHO_ARQUIVO)
print(f'📁 Abas disponíveis: {xlsx.sheet_names}')

# Carregar cada aba em um DataFrame separado
df_2022 = pd.read_excel(xlsx, sheet_name='PEDE2022')
df_2023 = pd.read_excel(xlsx, sheet_name='PEDE2023')
df_2024 = pd.read_excel(xlsx, sheet_name='PEDE2024')

print(f'\n📊 Dados carregados:')
print(f'   PEDE 2022: {df_2022.shape[0]:,} alunos, {df_2022.shape[1]} variáveis')
print(f'   PEDE 2023: {df_2023.shape[0]:,} alunos, {df_2023.shape[1]} variáveis')
print(f'   PEDE 2024: {df_2024.shape[0]:,} alunos, {df_2024.shape[1]} variáveis')

## 3. Visão Geral dos Dados

In [ ]:
def visao_geral(df, nome):
    """Exibe visão geral de um DataFrame"""
    print(f'\n{"="*60}')
    print(f'📋 {nome}')
    print(f'{"="*60}')
    print(f'\nDimensões: {df.shape[0]:,} linhas x {df.shape[1]} colunas')
    print(f'\nTipos de dados:')
    print(df.dtypes.value_counts())
    print(f'\nMemória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')
    
    # Valores nulos
    nulos = df.isnull().sum()
    nulos_pct = (nulos / len(df) * 100).round(2)
    nulos_df = pd.DataFrame({'Nulos': nulos, 'Percentual (%)': nulos_pct})
    nulos_df = nulos_df[nulos_df['Nulos'] > 0].sort_values('Nulos', ascending=False)
    
    if len(nulos_df) > 0:
        print(f'\n⚠️ Colunas com valores nulos (top 10):')
        print(nulos_df.head(10))
    else:
        print(f'\n✅ Nenhum valor nulo encontrado!')
    
    return nulos_df

nulos_2022 = visao_geral(df_2022, 'PEDE 2022')
nulos_2023 = visao_geral(df_2023, 'PEDE 2023')
nulos_2024 = visao_geral(df_2024, 'PEDE 2024')

In [ ]:
# Visualizar primeiras linhas de cada ano
print('\n📋 Primeiras linhas - PEDE 2022:')
df_2022.head(3)

In [ ]:
print('\n📋 Primeiras linhas - PEDE 2023:')
df_2023.head(3)

In [ ]:
print('\n📋 Primeiras linhas - PEDE 2024:')
df_2024.head(3)

## 4. Padronização e Limpeza dos Dados

In [ ]:
# Função para padronizar nomes de colunas
def padronizar_colunas(df, ano):
    """Padroniza nomes de colunas para facilitar análise"""
    df = df.copy()
    
    # Mapeamento de colunas para nomes padronizados
    mapeamento = {}
    
    for col in df.columns:
        col_lower = col.lower()
        
        # Identificar colunas de indicadores
        if col_lower == 'iaa' or col_lower.startswith('iaa'):
            mapeamento[col] = 'IAA'
        elif col_lower == 'ieg' or (col_lower.startswith('ieg') and 'destaque' not in col_lower):
            mapeamento[col] = 'IEG'
        elif col_lower == 'ips' or col_lower.startswith('ips'):
            mapeamento[col] = 'IPS'
        elif col_lower == 'ipp' or col_lower.startswith('ipp'):
            mapeamento[col] = 'IPP'
        elif col_lower == 'ida' or (col_lower.startswith('ida') and 'destaque' not in col_lower):
            mapeamento[col] = 'IDA'
        elif col_lower == 'ipv' or (col_lower.startswith('ipv') and 'destaque' not in col_lower):
            mapeamento[col] = 'IPV'
        elif col_lower == 'ian' or col_lower.startswith('ian'):
            mapeamento[col] = 'IAN'
        elif f'inde {ano}' in col_lower or f'inde_{ano}' in col_lower or col_lower == f'inde {ano[-2:]}':
            mapeamento[col] = 'INDE'
        elif 'defas' in col_lower:
            mapeamento[col] = 'DEFASAGEM'
        elif col_lower == 'fase' or col_lower.startswith('fase_'):
            mapeamento[col] = 'FASE'
        elif 'fase ideal' in col_lower:
            mapeamento[col] = 'FASE_IDEAL'
        elif f'pedra {ano}' in col_lower or f'pedra_{ano}' in col_lower or col_lower == f'pedra {ano[-2:]}':
            mapeamento[col] = 'PEDRA'
        elif 'atingiu pv' in col_lower or 'ponto_virada' in col_lower:
            mapeamento[col] = 'PONTO_VIRADA'
        elif col_lower == 'gênero' or col_lower == 'genero':
            mapeamento[col] = 'GENERO'
        elif 'ano ingresso' in col_lower or 'ano_ingresso' in col_lower:
            mapeamento[col] = 'ANO_INGRESSO'
        elif col_lower == 'idade' or f'idade {ano[-2:]}' in col_lower:
            mapeamento[col] = 'IDADE'
        elif col_lower == 'ra':
            mapeamento[col] = 'RA'
    
    # Aplicar mapeamento
    df = df.rename(columns=mapeamento)
    
    # Adicionar coluna de ano
    df['ANO'] = int(ano)
    
    return df

# Aplicar padronização
df_2022_pad = padronizar_colunas(df_2022, '2022')
df_2023_pad = padronizar_colunas(df_2023, '2023')
df_2024_pad = padronizar_colunas(df_2024, '2024')

print('✅ Colunas padronizadas!')
print(f'\nColunas 2022: {list(df_2022_pad.columns)[:15]}...')
print(f'Colunas 2023: {list(df_2023_pad.columns)[:15]}...')
print(f'Colunas 2024: {list(df_2024_pad.columns)[:15]}...')

In [ ]:
# Selecionar colunas principais para análise unificada
colunas_principais = ['RA', 'ANO', 'FASE', 'PEDRA', 'INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'DEFASAGEM', 'GENERO', 'IDADE', 'ANO_INGRESSO', 'PONTO_VIRADA']

def selecionar_colunas(df, colunas):
    """Seleciona apenas colunas existentes no DataFrame"""
    colunas_existentes = [c for c in colunas if c in df.columns]
    return df[colunas_existentes].copy()

df_2022_sel = selecionar_colunas(df_2022_pad, colunas_principais)
df_2023_sel = selecionar_colunas(df_2023_pad, colunas_principais)
df_2024_sel = selecionar_colunas(df_2024_pad, colunas_principais)

print('Colunas selecionadas por ano:')
print(f'2022: {list(df_2022_sel.columns)}')
print(f'2023: {list(df_2023_sel.columns)}')
print(f'2024: {list(df_2024_sel.columns)}')

In [ ]:
# Criar DataFrame unificado para análises comparativas
df_unificado = pd.concat([df_2022_sel, df_2023_sel, df_2024_sel], ignore_index=True)

# Padronizar valores de PEDRA
df_unificado['PEDRA'] = df_unificado['PEDRA'].replace({'Agata': 'Ágata'})

print(f'\n📊 DataFrame Unificado:')
print(f'   Total de registros: {len(df_unificado):,}')
print(f'   Colunas: {list(df_unificado.columns)}')
print(f'\nDistribuição por ano:')
print(df_unificado['ANO'].value_counts().sort_index())

## 5. Análise dos Indicadores

### 5.1 Estatísticas Descritivas dos Indicadores

In [ ]:
# Indicadores para análise
indicadores = ['INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN']

# Estatísticas por ano
stats_por_ano = df_unificado.groupby('ANO')[indicadores].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
print('📊 Estatísticas dos Indicadores por Ano:')
stats_por_ano

In [ ]:
# Visualização da evolução dos indicadores ao longo dos anos
fig, axes = plt.subplots(2, 4, figsize=(16, 10))
axes = axes.flatten()

for i, ind in enumerate(indicadores):
    ax = axes[i]
    
    # Boxplot por ano
    df_plot = df_unificado[['ANO', ind]].dropna()
    df_plot.boxplot(column=ind, by='ANO', ax=ax)
    
    ax.set_title(f'{ind}')
    ax.set_xlabel('Ano')
    ax.set_ylabel('Valor')

plt.suptitle('Distribuição dos Indicadores por Ano', fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig('indicadores_por_ano.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Evolução das médias dos indicadores
medias_por_ano = df_unificado.groupby('ANO')[indicadores].mean()

fig = go.Figure()

for ind in indicadores:
    fig.add_trace(go.Scatter(
        x=medias_por_ano.index,
        y=medias_por_ano[ind],
        mode='lines+markers',
        name=ind,
        line=dict(width=3),
        marker=dict(size=10)
    ))

fig.update_layout(
    title='Evolução das Médias dos Indicadores (2022-2024)',
    xaxis_title='Ano',
    yaxis_title='Média',
    legend_title='Indicador',
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

### 5.2 Distribuição das Pedras (Classificação INDE)

In [ ]:
# Distribuição das Pedras por ano
pedras_por_ano = df_unificado.groupby(['ANO', 'PEDRA']).size().unstack(fill_value=0)

# Ordenar pedras
ordem_pedras = ['Quartzo', 'Ágata', 'Ametista', 'Topázio']
pedras_existentes = [p for p in ordem_pedras if p in pedras_por_ano.columns]
pedras_por_ano = pedras_por_ano[pedras_existentes]

print('📊 Distribuição das Pedras por Ano:')
print(pedras_por_ano)
print('\nPercentual:')
print((pedras_por_ano.div(pedras_por_ano.sum(axis=1), axis=0) * 100).round(2))

In [ ]:
# Gráfico de barras empilhadas - Distribuição das Pedras
fig, ax = plt.subplots(figsize=(12, 6))

# Calcular percentuais
pedras_pct = pedras_por_ano.div(pedras_por_ano.sum(axis=1), axis=0) * 100

# Cores das pedras
cores = [CORES_PEDRAS.get(p, '#808080') for p in pedras_existentes]

pedras_pct.plot(kind='bar', stacked=True, ax=ax, color=cores, edgecolor='black', linewidth=0.5)

ax.set_title('Distribuição das Pedras por Ano (%)', fontsize=14, fontweight='bold')
ax.set_xlabel('Ano', fontsize=12)
ax.set_ylabel('Percentual (%)', fontsize=12)
ax.legend(title='Pedra', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

# Adicionar valores nas barras
for c in ax.containers:
    ax.bar_label(c, fmt='%.1f%%', label_type='center', fontsize=9)

plt.tight_layout()
plt.savefig('distribuicao_pedras.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.3 Análise da Defasagem (IAN)

In [ ]:
# Distribuição da defasagem por ano
defasagem_por_ano = df_unificado.groupby(['ANO', 'DEFASAGEM']).size().unstack(fill_value=0)

print('📊 Distribuição da Defasagem por Ano:')
print(defasagem_por_ano)

# Estatísticas da defasagem
print('\n📈 Estatísticas da Defasagem por Ano:')
print(df_unificado.groupby('ANO')['DEFASAGEM'].agg(['mean', 'median', 'std', 'min', 'max']))

In [ ]:
# Classificar defasagem em categorias
def classificar_defasagem(valor):
    if pd.isna(valor):
        return 'Não informado'
    elif valor <= -3:
        return 'Severamente defasado'
    elif valor <= -1:
        return 'Moderadamente defasado'
    elif valor == 0:
        return 'Adequado ao nível'
    else:
        return 'Acima do nível'

df_unificado['CATEGORIA_DEFASAGEM'] = df_unificado['DEFASAGEM'].apply(classificar_defasagem)

# Distribuição por categoria
cat_defasagem = df_unificado.groupby(['ANO', 'CATEGORIA_DEFASAGEM']).size().unstack(fill_value=0)

# Ordenar categorias
ordem_cat = ['Severamente defasado', 'Moderadamente defasado', 'Adequado ao nível', 'Acima do nível']
cat_existentes = [c for c in ordem_cat if c in cat_defasagem.columns]
cat_defasagem = cat_defasagem[cat_existentes]

print('📊 Distribuição por Categoria de Defasagem:')
print(cat_defasagem)
print('\nPercentual:')
print((cat_defasagem.div(cat_defasagem.sum(axis=1), axis=0) * 100).round(2))

In [ ]:
# Visualização da evolução da defasagem
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Histograma da defasagem por ano
ax1 = axes[0]
for ano in [2022, 2023, 2024]:
    dados = df_unificado[df_unificado['ANO'] == ano]['DEFASAGEM'].dropna()
    ax1.hist(dados, bins=range(-6, 5), alpha=0.5, label=str(ano), edgecolor='black')

ax1.set_title('Distribuição da Defasagem por Ano', fontsize=12, fontweight='bold')
ax1.set_xlabel('Defasagem (anos)')
ax1.set_ylabel('Frequência')
ax1.legend(title='Ano')
ax1.axvline(x=0, color='red', linestyle='--', label='Nível adequado')

# Gráfico 2: Evolução da média de defasagem
ax2 = axes[1]
media_defasagem = df_unificado.groupby('ANO')['DEFASAGEM'].mean()
ax2.plot(media_defasagem.index, media_defasagem.values, marker='o', linewidth=2, markersize=10, color='#2E86AB')
ax2.fill_between(media_defasagem.index, media_defasagem.values, alpha=0.3, color='#2E86AB')

ax2.set_title('Evolução da Média de Defasagem', fontsize=12, fontweight='bold')
ax2.set_xlabel('Ano')
ax2.set_ylabel('Média de Defasagem')
ax2.axhline(y=0, color='green', linestyle='--', label='Nível adequado')
ax2.legend()

# Adicionar valores nos pontos
for x, y in zip(media_defasagem.index, media_defasagem.values):
    ax2.annotate(f'{y:.2f}', (x, y), textcoords='offset points', xytext=(0, 10), ha='center', fontsize=11)

plt.tight_layout()
plt.savefig('evolucao_defasagem.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.4 Matriz de Correlação dos Indicadores

In [ ]:
# Calcular matriz de correlação
indicadores_corr = ['INDE', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'DEFASAGEM']
indicadores_existentes = [i for i in indicadores_corr if i in df_unificado.columns]

corr_matrix = df_unificado[indicadores_existentes].corr()

# Heatmap de correlação
fig, ax = plt.subplots(figsize=(10, 8))

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r', 
            center=0, square=True, linewidths=0.5, ax=ax,
            cbar_kws={'shrink': 0.8, 'label': 'Correlação'})

ax.set_title('Matriz de Correlação dos Indicadores', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('matriz_correlacao.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n📊 Correlações mais fortes com INDE:')
corr_inde = corr_matrix['INDE'].drop('INDE').sort_values(ascending=False)
print(corr_inde)

## 6. Análise por Fase do Programa

In [ ]:
# Distribuição dos alunos por fase
print('📊 Distribuição por Fase (2022):')
print(df_2022['Fase'].value_counts().sort_index())

print('\n📊 Distribuição por Fase (2023):')
print(df_2023['Fase'].value_counts())

print('\n📊 Distribuição por Fase (2024):')
print(df_2024['Fase'].value_counts().head(15))

In [ ]:
# Análise de indicadores por Pedra
indicadores_analise = ['INDE', 'IDA', 'IEG', 'IAA', 'IPS', 'IPP', 'IPV', 'IAN']

# Médias por Pedra
medias_por_pedra = df_unificado.groupby('PEDRA')[indicadores_analise].mean()

# Ordenar por INDE
ordem_pedras = ['Quartzo', 'Ágata', 'Ametista', 'Topázio']
medias_por_pedra = medias_por_pedra.reindex([p for p in ordem_pedras if p in medias_por_pedra.index])

print('📊 Médias dos Indicadores por Pedra:')
medias_por_pedra.round(2)

In [ ]:
# Visualização radar dos indicadores por Pedra
from math import pi

# Preparar dados para radar
categorias = indicadores_analise
N = len(categorias)

# Criar ângulos
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

for pedra in medias_por_pedra.index:
    valores = medias_por_pedra.loc[pedra].values.tolist()
    valores += valores[:1]
    
    ax.plot(angles, valores, linewidth=2, linestyle='solid', label=pedra, 
            color=CORES_PEDRAS.get(pedra, '#808080'))
    ax.fill(angles, valores, alpha=0.1, color=CORES_PEDRAS.get(pedra, '#808080'))

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categorias, fontsize=11)
ax.set_title('Perfil dos Indicadores por Pedra', fontsize=14, fontweight='bold', y=1.08)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.savefig('radar_indicadores_pedra.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Salvando Dados Processados

In [ ]:
# Salvar DataFrame unificado para uso posterior
df_unificado.to_csv('dados_unificados_passos_magicos.csv', index=False)
print('✅ Dados salvos em: dados_unificados_passos_magicos.csv')

# Resumo final
print(f'\n📊 Resumo do Dataset Unificado:')
print(f'   Total de registros: {len(df_unificado):,}')
print(f'   Período: 2022 a 2024')
print(f'   Colunas: {len(df_unificado.columns)}')
print(f'\n   Distribuição por ano:')
print(df_unificado['ANO'].value_counts().sort_index())

---

## 📝 Próximos Passos

1. **Notebook 2:** Responder às 11 perguntas de negócio com análises detalhadas
2. **Notebook 3:** Desenvolver modelo preditivo de risco de defasagem
3. **Streamlit:** Criar aplicação para deploy do modelo

---